In [1]:
import os

In [2]:
%pwd

'/home/highfrezh/Desktop/AI-ML-ENG/DL PROJECTS/end-to-end-brain-tumor-classification-using-MRI-Dataset/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/highfrezh/Desktop/AI-ML-ENG/DL PROJECTS/end-to-end-brain-tumor-classification-using-MRI-Dataset'

In [5]:
# import dagshub
# dagshub.init(repo_owner='bappymalik4161', repo_name='ML-Pipeline-using-DVC', mlflow=True)

In [6]:
from dataclasses import dataclass
from pathlib import Path
import pickle


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    val_data_path: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/braintumor.h5",
            val_data_path="artifacts/split_data",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [9]:
import tensorflow as tf
from urllib.parse import urlparse

2024-12-30 00:48:10.281708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-30 00:48:10.316536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-30 00:48:10.325132: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-30 00:48:10.350150: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 00:48:12.562919: W tensorflow/compiler/tf2

In [10]:
# model = tf.keras.models.load_model("artifacts/training/braintumor_vgg.h5")

In [11]:
class Evaluation:
    def __init__(self, eval_config: EvaluationConfig):
        self.config = eval_config

    def load_saved_data_pickle(self):
        """
        Load the saved validation datasets.
        """
        with open(os.path.join(self.config.val_data_path, "val_data.pkl"), "rb") as f:
            X_val, Y_val = pickle.load(f)

            print(X_val.shape)
            print(Y_val.shape)

        return X_val, Y_val

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        print(f"Loading model from {path}")
        return tf.keras.models.load_model(path)
    

    def evaluation(self,X_val, Y_val):
        model = self.load_model(self.config.path_of_model)
        print("Model loaded successfully")
        self.score = model.evaluate(X_val, Y_val)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    X_val, Y_val = evaluation.load_saved_data_pickle()
    evaluation.evaluation(X_val, Y_val)
    evaluation.save_score()

except Exception as e:
   raise e

[2024-12-30 00:48:16,837: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-30 00:48:16,847: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-30 00:48:16,850: INFO: common: created directory at: artifacts]
(1405, 224, 224, 3)
(1405, 4)
Loading model from artifacts/training/braintumor.h5


2024-12-30 00:48:20.598898: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25690112 exceeds 10% of free system memory.
2024-12-30 00:48:20.633150: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25690112 exceeds 10% of free system memory.
2024-12-30 00:48:20.652886: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25690112 exceeds 10% of free system memory.


[2024-12-30 00:48:21,064: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


2024-12-30 00:48:21.003136: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25690112 exceeds 10% of free system memory.


Model loaded successfully


2024-12-30 00:48:23.102897: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 845967360 exceeds 10% of free system memory.


44/44 ━━━━━━━━━━━━━━━━━━━━ 537s 12s/step - accuracy: 0.9720 - loss: 0.0648
[2024-12-30 00:57:21,269: INFO: common: json file saved at: scores.json]


In [13]:
model = tf.keras.models.load_model('../artifacts/training/new_model.h5')

# Get the input shape
input_shape = model.input_shape
print(f"Model was trained with input shape: {input_shape}")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '../artifacts/training/new_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import tensorflow as tf
print(tf.__version__)